In [17]:
# k-nearest neighbors on the Iris Flowers Dataset
from random import seed
from random import randrange
from csv import reader
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import time 


In [18]:
import numpy as np
import json
from web3 import Web3
import time
ganache_url = "HTTP://127.0.0.1:8545"
# ropsten_url ="https://ropsten.infura.io/v3/6072fdc3b95c407caa681f63ebaf72b8"
web3 = Web3(Web3.HTTPProvider(ganache_url))
# web3 = Web3(Web3.HTTPProvider(ropsten_url))
web3.isConnected()

True

In [19]:
abi = json.loads('[ { "inputs": [ { "internalType": "int256[]", "name": "a", "type": "int256[]" }, { "internalType": "int256[]", "name": "b", "type": "int256[]" } ], "name": "euc_distance", "outputs": [ { "internalType": "int256", "name": "", "type": "int256" } ], "stateMutability": "nonpayable", "type": "function" }, { "inputs": [ { "internalType": "uint256[]", "name": "label", "type": "uint256[]" } ], "name": "get_predict", "outputs": [], "stateMutability": "nonpayable", "type": "function" }, { "inputs": [], "name": "set_dist", "outputs": [ { "internalType": "int256", "name": "", "type": "int256" } ], "stateMutability": "view", "type": "function" }, { "inputs": [], "name": "set_pos", "outputs": [ { "internalType": "uint256[]", "name": "", "type": "uint256[]" } ], "stateMutability": "view", "type": "function" }, { "inputs": [], "name": "set_predict", "outputs": [ { "internalType": "uint256", "name": "", "type": "uint256" } ], "stateMutability": "view", "type": "function" }, { "inputs": [], "name": "set_sort_dist", "outputs": [ { "internalType": "uint256[]", "name": "", "type": "uint256[]" } ], "stateMutability": "view", "type": "function" }, { "inputs": [ { "internalType": "uint256[]", "name": "data", "type": "uint256[]" }, { "internalType": "uint256[]", "name": "pos", "type": "uint256[]" } ], "name": "sort_dist", "outputs": [ { "internalType": "uint256[]", "name": "", "type": "uint256[]" }, { "internalType": "uint256[]", "name": "", "type": "uint256[]" } ], "stateMutability": "nonpayable", "type": "function" } ]')
cont_address = "0xc3facDa10F2D9170D5472bdC7e09A673D96BbED3"
contract = web3.eth.contract(address=cont_address,abi=abi)

In [20]:
account1 = "0xacF9197258fEB28d7c2333c0625841Cc9A33A794"
private_key1 = "edbc15277f4f7bae99b111c25be1a67372186cdcc972d8257598637ce54fda42"
account2 = "0xcbd7C06c8d7988A62e7a8a84D59d748D5A790db1"
private_key2="b5b736f3ca08acf6d8acd03eadeb48e164544b8f9f40a08c83fdfc89a1e565b2"
account3 ="0x27bD68B531761c280B41FC8f4f2cE2346F6ce45b"
private_key3 ="07bd02e881fe8949eef1033f22a30a7ffb7b5be78ed11966483df03d6897e123" 

In [21]:
#reset position array
def reset_pos(x, account, private_key,contract):
    nonce = web3.eth.getTransactionCount(account)
    tx_store = contract.functions.reset_pos(
        x
         ).buildTransaction({
        'gas': 3000000,
        'gasPrice': web3.toWei('10', 'gwei'),
        'from': account,
        'nonce': nonce
        })

    signed_txn = web3.eth.account.signTransaction(tx_store, private_key=private_key)
    start_time = time.time()
    tx_hash = web3.eth.sendRawTransaction(signed_txn.rawTransaction)
    tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)
    end_time = time.time()
    return(tx_receipt.gasUsed)
#     print("the stored number is ", contract.functions.retrieve().call())

In [22]:
#eucledean distance function from contract
def get_euc_distance(x,y, account, private_key,contract):
    nonce = web3.eth.getTransactionCount(account)
    tx_store = contract.functions.euc_distance(
        x,y
         ).buildTransaction({
        'gas': 3000000,
        'gasPrice': web3.toWei('10', 'gwei'),
        'from': account,
        'nonce': nonce
        })

    signed_txn = web3.eth.account.signTransaction(tx_store, private_key=private_key)
    start_time = time.time()
    tx_hash = web3.eth.sendRawTransaction(signed_txn.rawTransaction)
    tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)
    end_time = time.time()
    return(tx_receipt.gasUsed)


In [23]:
#sort distance from contract
def get_sorted_distance(x,y, account, private_key,contract):
    nonce = web3.eth.getTransactionCount(account)
    tx_store = contract.functions.sort_dist(
        x,y
         ).buildTransaction({
        'gas': 8000000,
        'gasPrice': web3.toWei('10', 'gwei'),
        'from': account,
        'nonce': nonce
        })

    signed_txn = web3.eth.account.signTransaction(tx_store, private_key=private_key)
    tx_hash = web3.eth.sendRawTransaction(signed_txn.rawTransaction)
    tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)
    return(tx_receipt.gasUsed)

In [24]:
#predict position
def get_predict(x, account, private_key,contract):
    nonce = web3.eth.getTransactionCount(account)
    tx_store = contract.functions.get_predict(
        x
         ).buildTransaction({
        'gas': 80000000,
        'gasPrice': web3.toWei('10', 'gwei'),
        'from': account,
        'nonce': nonce
        })

    signed_txn = web3.eth.account.signTransaction(tx_store, private_key=private_key)
    tx_hash = web3.eth.sendRawTransaction(signed_txn.rawTransaction)
    tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)
    return(tx_receipt.gasUsed)


In [25]:

# import some data to play with
iris = datasets.load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.30, random_state=42)
X_train = (X_train * 10).astype(int)
X_test = (X_test * 10).astype(int)

In [26]:
a = X_train[0].tolist()
sample = X_test[0].tolist()

In [27]:
print(a,sample)

[55, 24, 37, 10] [61, 28, 47, 12]


In [28]:
predicted =[] 
total_dist_gas=[]
total_sorted_gas=[]
total_pred_gas=[]

dist_arr =np.zeros((len(y_test),len(y_train)))
dist_arr.shape

(45, 105)

In [29]:
# dist_arr[0]=x

In [30]:
# dist_arr[0]

In [31]:

for i in range(len(y_test)):  
    print(i)
    dist = []
    
    dist_gas=[]
    sample = X_test[i].tolist()
    for j in range(X_train.shape[0]):
        a = X_train[j].tolist()
        dist_gas.append(get_euc_distance(a,sample,account3, private_key3,contract))
        dist.append(contract.functions.set_dist().call())
    x = np.array(dist)
    dist_arr[i] = x
#     print(dist_arr[i])
    total_dist_gas.append(sum(dist_gas))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44


In [32]:
dist_gas

[31511,
 31511,
 31511,
 31511,
 31620,
 31511,
 31620,
 31511,
 31075,
 31620,
 31402,
 31075,
 31075,
 26875,
 31511,
 31620,
 31184,
 31184,
 31075,
 31511,
 31075,
 31511,
 31620,
 31184,
 31511,
 31511,
 31075,
 31511,
 31511,
 31511,
 31511,
 31511,
 31511,
 31075,
 31402,
 31620,
 31184,
 31184,
 31511,
 31511,
 31184,
 31511,
 31075,
 31075,
 31511,
 31402,
 31620,
 31511,
 31511,
 31729,
 31511,
 31293,
 31293,
 31511,
 31511,
 31184,
 31075,
 31293,
 31511,
 31511,
 31184,
 31620,
 31620,
 31293,
 31511,
 31511,
 31511,
 27311,
 31620,
 30966,
 31511,
 31402,
 31511,
 31402,
 31511,
 31511,
 31402,
 31511,
 31511,
 30748,
 31402,
 31620,
 31511,
 31075,
 31402,
 31620,
 27311,
 31075,
 31620,
 31184,
 31511,
 31620,
 31511,
 31511,
 31511,
 31511,
 31511,
 31511,
 31511,
 31075,
 31511,
 31511,
 31184,
 31402,
 31511]

In [33]:
a = np.array(dist_gas)
np.savetxt('distance_gas.csv', a, delimiter=',')

In [34]:
dist_arr

array([[12.,  5., 10., ..., 38.,  7., 18.],
       [25., 37., 42., ...,  5., 27., 48.],
       [41., 24., 20., ..., 65., 36., 12.],
       ...,
       [17.,  6.,  9., ..., 39., 12., 14.],
       [28., 42., 46., ...,  4., 31., 52.],
       [26., 40., 44., ...,  5., 29., 51.]])

In [47]:


datapoints=np.arange(105)
data = datapoints.tolist()
# Gas_mean = [29654,80656,108084,136223,164402,192607,220863,249146,277481,305854,591689]
# Gas_variance=[89761,116422,160750,205789,250868,295973,341129,386312,431547,476820,931655]

gas = dist_gas

# bx = plt.subplots()
fig = plt.figure(figsize=(10, 10))
plt.plot(datapoints,gas,marker='o', color='b',label ='Mean',linewidth=2.0)
# plt.plot(datapoints,Gas_variance,marker='o', color='r',label = 'Variance',linewidth=2.0)
plt.ticklabel_format(style='plain')
leg = plt.legend()
fig.suptitle('Mean  And Vairiance Computational Cost Scalability',fontsize=12, )
plt.xlabel('Number of datapoints for training',fontsize=12)
plt.ylabel('Gas Usage with Gas Price 10Gwei',fontsize=12)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
# plt.set(xlabel='Data points', ylabel='Gas Use',
#        title='Gas use versus Mean & variance Calculation')
# plt.show()
# plt.savefig('cost.pdf')

AttributeError: module 'numpy' has no attribute 'arrange'

In [35]:
# print(total_dist_gas)
# print(sort_dis_gas)
# print(pred_gas)

In [36]:
# print(y_train[64])

In [37]:
# a= list(range(10))
# b= list(range(10))
# c = 0
# d = len(a)


In [38]:
# sort_dis_gas = get_sorted_distance(a,b,c,d,account3, private_key3,contract)

In [39]:
# ylabel = list(range(10))

In [40]:
# web3.eth.get_block('latest')
# pred_gas = get_predict(ylabel, account3, private_key3,contract)

In [41]:
# predicted

In [42]:
# y_test

In [43]:
# from sklearn.metrics import accuracy_score
# accuracy_score(y_test, pred)

In [44]:
# neigh = KNeighborsClassifier(n_neighbors=3)
# >>> neigh.fit(X, y)
# KNeighborsClassifier(...)
# >>> print(neigh.predict([[1.1]]))
# [0]
# >>> print(neigh.predict_proba([[0.9]]))